### TARS DSPy Laboratory
#### Wilton Beltré

#### Simple Predict

In [124]:
from dspy import OpenAI
from dspy import Predict, settings, ChainOfThought, ChainOfThoughtWithHint, MultiChainComparison, Prediction, ProgramOfThought, ReAct
from rich import print
from dotenv import dotenv_values

secret = dotenv_values('../../.secret')
gpt_turbo = OpenAI('gpt-3.5-turbo', api_key=secret['OPEN_AI_API_KEY'], model_type="chat")
settings.configure(lm=gpt_turbo)

from dspy import GROQ

# llama3-70b-8192
# groq = GROQ(model='llama3-70b-8192', api_key='')
# settings.configure(lm=groq)



In [129]:
chat = Predict("context, question -> answer")
chat(question="Yes, fisher").with_inputs("context")

Prediction(
    context='Context: Fishing is a popular recreational activity enjoyed by many people around the world.',
    answer='No, I am not a fisher.'
)

In [130]:
import dspy 

lm = dspy.HFModel(model="google/flan-t5-base", )


ModuleNotFoundError: You need to install Hugging Face transformers library to use HF models.

In [21]:
pred = Predict("question -> answer", temperature=0.3)
response = pred(question="Why does popcorn pop?")
print(response)

Prediction(
    answer='Question: Why does popcorn pop?\nAnswer: Popcorn pops because water molecules inside the kernel turn 
into steam when heated, building up pressure until the kernel explodes, turning inside out.'
)

In [10]:
predict = Predict("question -> answer_as_json")
print(predict)

Predict(StringSignature(question -> answer_as_json
    instructions='Given the fields `question`, produce the fields `answer_as_json`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Question:', 'desc': '${question}'})
    answer_as_json = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix':
'Answer As Json:', 'desc': '${answer_as_json}'})
))

In [13]:
response = predict(question="Provide 3 method of statistical inference")
print(response.answer_as_json)

{
  "method_1": "Hypothesis testing",
  "method_2": "Confidence intervals",
  "method_3": "Regression analysis"
}

#### Chain Of Thought - Technique (reasoning)

In [19]:
cot_qa = ChainOfThought("question -> answer", temperature=0.3)
print(cot_qa)

ChainOfThought(StringSignature(question -> answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 
'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 
'Answer:', 'desc': '${answer}'})
))

In [21]:
response = cot_qa(question="What year was Cristobal Colon imprisoned and won the novel award?")

In [22]:
print(response)

Prediction(
    rationale='produce the answer. We know that Cristobal Colon, also known as Christopher Columbus, was imprisoned
in 1500 in Spain. However, he did not win a novel award because he was not a writer.',
    answer='Cristobal Colon was imprisoned in 1500, but he did not win a novel award as he was not a writer.'
)

In [23]:
gpt_turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: What year was Cristobal Colon imprisoned and won the novel award?
Reasoning: Let's think step by step in order to produce the answer. We know that Cristobal Colon, also known as Christopher Columbus, was imprisoned in 1500 in Spain. However, he did not win a novel award because he was not a writer. 
Answer: Cristobal Colon was imprisoned in 1500, but he did not win a novel award as he was not a writer.





"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: What year was Cristobal Colon imprisoned and won the novel award?\nReasoning: Let's think step by step in order to\x1b produce the answer. We know that Cristobal Colon, also known as Christopher Columbus, was imprisoned in 1500 in Spain. However, he did not win a novel award because he was not a writer. \nAnswer: Cristobal Colon was imprisoned in 1500, but he did not win a novel award as he was not a writer.\x1b\n\n\n"

#### Chain Of Thought with Hint - Technique (reasoning with hints)

In [26]:
question = "In the legal framework, how many weekly working hours are established for the call center industry?"
hint = "Call centers are nearshore industries based in the Caribbean and their clients are in the United States."

coth_qa = ChainOfThoughtWithHint("question -> answer")
response = coth_qa(question=question, hint=hint)
print(response)

Prediction(
    rationale="Question: In the legal framework, how many weekly working hours are established for the call center 
industry?\n\nReasoning: Let's think step by step in order to determine the answer. Call centers are often subject 
to specific regulations due to their unique operating hours and client base.",
    answer='The legal framework typically establishes 40-hour workweeks for the call center industry.'
)

In [27]:
gpt_turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Hint: ${hint}

Answer: ${answer}

---

Question: In the legal framework, how many weekly working hours are established for the call center industry?

Hint: Call centers are nearshore industries based in the Caribbean and their clients are in the United States.

Answer: Question: In the legal framework, how many weekly working hours are established for the call center industry?

Reasoning: Let's think step by step in order to determine the answer. Call centers are often subject to specific regulations due to their unique operating hours and client base. 

Hint: Call centers are nearshore industries based in the Caribbean and their clients are in the United States.

Answer: The legal framework typically establishes 40-hour workweeks for the call center industry.





"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\n\nHint: ${hint}\n\nAnswer: ${answer}\n\n---\n\nQuestion: In the legal framework, how many weekly working hours are established for the call center industry?\n\nHint: Call centers are nearshore industries based in the Caribbean and their clients are in the United States.\n\nAnswer:\x1b Question: In the legal framework, how many weekly working hours are established for the call center industry?\n\nReasoning: Let's think step by step in order to determine the answer. Call centers are often subject to specific regulations due to their unique operating hours and client base. \n\nHint: Call centers are nearshore industries based in the Caribbean and their clients are in the United States.\n\nAnswer: The legal framework typically establishes 40-hour workweeks for the call center indust

#### Multi Chain Comparison - technique of "students" and final reasonable answer

In [31]:
completions = [
    Prediction(rationale="Yes, when I visit my brother I can identify.", answer="personality identification"),
    Prediction(rationale="a person who is migrating from one country to another", answer="personality migration"),
    Prediction(rationale="bad friend", answer="fake personality"),
]

multichain = MultiChainComparison("question -> answer")

response = multichain(question="distinguishes an individual from others by describing feelings, thinking, and behaviors", completions=completions)

print(response)

Prediction(
    rationale='correct the answers. Personality distinguishes an individual from others by describing feelings, 
thinking, and behaviors. It is the unique pattern of characteristics that defines an individual.',
    answer='Personality'
)

In [32]:
gpt_turbo.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}

Student Attempt #1: ${reasoning attempt}

Student Attempt #2: ${reasoning attempt}

Student Attempt #3: ${reasoning attempt}

Accurate Reasoning: Thank you everyone. Let's now holistically ${corrected reasoning}

Answer: ${answer}

---

Question: distinguishes an individual from others by describing feelings, thinking, and behaviors

Student Attempt #1: «I'm trying to Yes, when I visit my brother I can identify. I'm not sure but my prediction is personality identification»

Student Attempt #2: «I'm trying to a person who is migrating from one country to another I'm not sure but my prediction is personality migration»

Student Attempt #3: «I'm trying to bad friend I'm not sure but my prediction is fake personality»

Accurate Reasoning: Thank you everyone. Let's now holistically correct the answers. Personality distinguishes an individual from others by describing feeli

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nStudent Attempt #1: ${reasoning attempt}\n\nStudent Attempt #2: ${reasoning attempt}\n\nStudent Attempt #3: ${reasoning attempt}\n\nAccurate Reasoning: Thank you everyone. Let's now holistically ${corrected reasoning}\n\nAnswer: ${answer}\n\n---\n\nQuestion: distinguishes an individual from others by describing feelings, thinking, and behaviors\n\nStudent Attempt #1: «I'm trying to Yes, when I visit my brother I can identify. I'm not sure but my prediction is personality identification»\n\nStudent Attempt #2: «I'm trying to a person who is migrating from one country to another I'm not sure but my prediction is personality migration»\n\nStudent Attempt #3: «I'm trying to bad friend I'm not sure but my prediction is fake personality»\n\nAccurate Reasoning: Thank you everyone. Let's now holistically\x1b correct the answers. Personality distinguishes an indivi

#### Program of Thought (logical and math reasoning)

In [39]:
pot = ProgramOfThought("question -> answer")
question = """From a bag containing red and blue balls, 10 each, 2 balls are drawn at random. 
Find the probability that one of them is red and the other is blue. """
response = pot(question=question)
print(response)

Error in code execution
Error in code execution


Prediction(
    rationale='produce the answer. We first calculate the total ways to choose 2 balls out of 20, which is 20 * 19 
/ 2. Then, we determine the desired ways to choose one red ball and one blue ball, which is 10 * 10. Finally, we 
divide the desired ways by the total ways to get the probability.',
    answer='0.5263157894736842'
)

In [41]:
gpt_turbo.inspect_history(n=1)




Given the final code `question`, `final_generated_code`, `code_output`, provide the final `answer`.

---

Follow the following format.

Question: ${question}

Code: python code that answers the question

Code Output: output of previously-generated python code

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Question: From a bag containing red and blue balls, 10 each, 2 balls are drawn at random. Find the probability that one of them is red and the other is blue.

Code:
total_ways = 20 * 19 / 2  # Total ways to choose 2 balls out of 20
desired_ways = 10 * 10  # 10 ways to choose a red ball and 10 ways to choose a blue ball
answer = desired_ways / total_ways
answer

Code Output: 0.5263157894736842

Reasoning: Let's think step by step in order to produce the answer. We first calculate the total ways to choose 2 balls out of 20, which is 20 * 19 / 2. Then, we determine the desired ways to choose one red ball and one blue ball, which

"\n\n\nGiven the final code `question`, `final_generated_code`, `code_output`, provide the final `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nCode: python code that answers the question\n\nCode Output: output of previously-generated python code\n\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nQuestion: From a bag containing red and blue balls, 10 each, 2 balls are drawn at random. Find the probability that one of them is red and the other is blue.\n\nCode:\ntotal_ways = 20 * 19 / 2  # Total ways to choose 2 balls out of 20\ndesired_ways = 10 * 10  # 10 ways to choose a red ball and 10 ways to choose a blue ball\nanswer = desired_ways / total_ways\nanswer\n\nCode Output: 0.5263157894736842\n\nReasoning: Let's think step by step in order to\x1b produce the answer. We first calculate the total ways to choose 2 balls out of 20, which is 20 * 19 / 2. Then, we determine the desired ways to choose o

#### ReAct (reasoning and acting)


In [108]:
import dspy
from dspy import ReAct, Signature
from rich import print

# Define a simple signature for basic question answering
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Pass signature to ReAct module
react_module = dspy.ReAct(BasicQA)

# Call the ReAct module on a particular input
question = 'Aside from the Apple Remote, what other devices can control the program Apple Remote was originally designed to interact with?'
question = 'What is the real name of Batman?'
result = react_module(question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after ReAct process): {result.answer}")

Question: What is the real name of Batman?

Final Predicted Answer (after ReAct process):

In [113]:
agent = dspy.ReAct("question -> answer")
agent(question="What's up!")

Prediction(
    observations=[],
    answer='Hey there! How can I help you today?'
)

In [114]:
agent(question="Could you help me with some basic math?")

Prediction(
    observations=['Failed to parse action. Bad formatting or incorrect action name.', 'Failed to parse action. Bad formatting or incorrect action name.', 'Failed to parse action. Bad formatting or incorrect action name.', 'Failed to parse action. Bad formatting or incorrect action name.', 'Failed to parse action. Bad formatting or incorrect action name.'],
    answer=''
)

In [117]:
gpt_turbo.inspect_history(n=10)




You will be given `question` and you will respond with `answer`.

To do this, you will interleave Thought, Action, and Observation steps.

Thought can reason about the current situation, and Action can be the following types:

(1) Search[query], which takes a search query and returns one or more potentially relevant passages from a corpus
(2) Finish[answer], which returns the final `answer` and finishes the task

---

Follow the following format.

Question: ${question}
Thought 1: next steps to take based on last observation
Action 1: always either Search[query] or, when done, Finish[answer]

---

Question: What's up!
Thought 1: I should respond with a casual greeting.
Action 1: Finish[Hey there! How can I help you today?]





You will be given `question` and you will respond with `answer`.

To do this, you will interleave Thought, Action, and Observation steps.

Thought can reason about the current situation, and Action can be the following types:

(1) Search[query], which takes a 

"\n\n\nYou will be given `question` and you will respond with `answer`.\n\nTo do this, you will interleave Thought, Action, and Observation steps.\n\nThought can reason about the current situation, and Action can be the following types:\n\n(1) Search[query], which takes a search query and returns one or more potentially relevant passages from a corpus\n(2) Finish[answer], which returns the final `answer` and finishes the task\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nThought 1: next steps to take based on last observation\nAction 1: always either Search[query] or, when done, Finish[answer]\n\n---\n\nQuestion: What's up!\nThought 1:\x1b I should respond with a casual greeting.\nAction 1: Finish[Hey there! How can I help you today?]\x1b\n\n\n\n\n\nYou will be given `question` and you will respond with `answer`.\n\nTo do this, you will interleave Thought, Action, and Observation steps.\n\nThought can reason about the current situation, and Action can be the followin

In [110]:
print(result)

Prediction(
    observations=['Failed to parse action. Bad formatting or incorrect action name.', 'Failed to parse action. Bad 
formatting or incorrect action name.', 'Failed to parse action. Bad formatting or incorrect action name.', 'Failed 
to parse action. Bad formatting or incorrect action name.', 'Failed to parse action. Bad formatting or incorrect 
action name.'],
    answer=''
)

In [107]:
gpt_turbo(question)

['Some other devices that can control the program Apple Remote was originally designed to interact with include:\n\n1. iPhone or iPad with the Apple Remote app\n2. Apple Watch with the Apple Remote app\n3. Mac computer with the Apple Remote app or built-in remote control functionality\n4. Universal remote controls that are compatible with Apple devices\n5. Third-party remote control apps that are compatible with Apple devices']

### Data & Metrics

In [9]:
from dspy import Example

ex1 = Example(question="What's position are open now?", answer="Agent Call Center", hint="Consult data base for updates on position jobs.", table_ref="hr_jobs")

print(ex1)

Example({'question': 'Whats position are open now?', 'answer': 'Agent Call Center', 'hint': 'Consult data base for 
updates on position jobs.', 'table_ref': 'hr_jobs'}) (input_keys=None)

In [10]:
ex2 = Example(question="According to company rules, what is the most valuable employee?", answer="Sales Agent").with_inputs("company_rules")
print(ex2)

Example({'question': 'According to company rules, what is the most valuable employee?', 'answer': 'Sales Agent'}) 
(input_keys={'company_rules'})

In [16]:
ex2.inputs(), ex2.labels()

(Example({}) (input_keys=None),
 Example({'question': 'According to company rules, what is the most valuable employee?', 'answer': 'Sales Agent'}) (input_keys=None))

In [17]:
from dspy.datasets import HotPotQA

hpqa = HotPotQA(train_seed=1, train_size=20, eval_seed=2, dev_size=5, test_size=3)


Generating test split: 100%|██████████| 7405/7405 [00:03<00:00, 2105.45 examples/s]
/Users/beltre.wilton/miniforge3/envs/tars_env/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [31]:
hpqa.dev


[Example({'question': 'Angela Diniz was born in Belo Horizonte which is the capital of which Brazilian state?', 'answer': 'Minas Gerais', 'gold_titles': {'Ângela Diniz', 'Belo Horizonte'}}) (input_keys=None),
 Example({'question': 'Which Indian symphonic conductor serves on the Anotonin Dvorak Music Festival named after the world-renowned Czech composer?', 'answer': 'Debashish Chaudhuri', 'gold_titles': {'Antonín Dvořák', 'Debashish Chaudhuri'}}) (input_keys=None),
 Example({'question': 'When was the American comedian, musician and Tony Award-nominated actor that headlined at Carolines on Broadway born?', 'answer': '1971', 'gold_titles': {'Carolines on Broadway', 'Stephen Lynch (musician)'}}) (input_keys=None),
 Example({'question': ' Suzana S. Drobnjaković Ponti acted in a film loosely based on a book by who?', 'answer': 'Danny Wallace', 'gold_titles': {'Yes Man (film)', 'Sasha Alexander'}}) (input_keys=None),
 Example({'question': 'Kentucky is located in a region of america which con

#### Metrics

In [44]:
from dspy import Signature, InputField, OutputField
from dspy.evaluate import answer_exact_match, answer_exact_match_str, answer_passage_match
from dspy import Example


ex1 = Example(question="What about sales?", answer="It's about closed sales.")
ex2 = Example(question="What about sales?", answer="It's about secured sales.")

print(answer_exact_match(ex1, ex2, frac=0.5))

print(answer_exact_match(ex1, ex2, frac=1.0))

True

False

In [48]:
# simple metric
def validate_ans(example: Example, pred: Example, trace=None):
    return example.answer.lower() == pred.answer.lower()

# multiple checks
def validate_context_and_answer(example: Example, pred: Example, contexts: list, trace=None):
    answer_match = example.answer.lower() == pred.answer.lower()

    context_match = any((pred.answer.lower() in c) for c in contexts)

    if trace is None:
        return (answer_match + context_match) / 2.0
    else:
        return answer_match and context_match
    


class Assess(Signature):
    """Assess the quality of a tweet along the specified dimension."""

    assessed_text = InputField()
    assesed_question = InputField()
    assessed_answer = OutputField(desc="Yes or Not")


model_pred = Predict("question -> answer")
assess_pred = Predict(Assess)


In [50]:
from dspy import context

# metric
def metric(gold: Example, pred: Example, trace=None):
    question, answer, gen_answer = gold.question, gold.answer, pred.answer

    engaging = "Is the assessed text self-contained, information?"
    correct = f"The text should answer `{question}` with `{answer}`. Does the assessed text contain this answer?"

    with context(lm=gpt_turbo):
        correct = assess_pred(assessed_text=gen_answer, assesed_question=correct)
        engaging = assess_pred(assessed_text=gen_answer, assesed_question=engaging)

    correct, engaging = [m.assessed_answer.lower() == "yes" for m in [correct, engaging]]
    score = (correct + engaging) if correct and (len(gen_answer) <= 280) else 0

    if trace is not None: return score >= 2

    return score / 2.0



In [58]:
print(hpqa.train[0])

output = model_pred(question=hpqa.train[0].question)
print(output)

# simple metric
print(validate_ans(hpqa.train[0], output))

# defined metric
print(metric(hpqa.train[0], output))

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van 
Zandt'}) (input_keys=None)

Prediction(
    answer='Bob Dylan'
)

False

0.0

In [60]:
gpt_turbo.inspect_history(n=3)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Answer: ${answer}

---

Question: At My Window was released by which American singer-songwriter?
Answer: Bob Dylan





Assess the quality of a tweet along the specified dimension.

---

Follow the following format.

Assessed Text: ${assessed_text}
Assesed Question: ${assesed_question}
Assessed Answer: Yes or Not

---

Assessed Text: Bob Dylan
Assesed Question: The text should answer `At My Window was released by which American singer-songwriter?` with `John Townes Van Zandt`. Does the assessed text contain this answer?
Assessed Answer: No





Assess the quality of a tweet along the specified dimension.

---

Follow the following format.

Assessed Text: ${assessed_text}
Assesed Question: ${assesed_question}
Assessed Answer: Yes or Not

---

Assessed Text: Bob Dylan
Assesed Question: Is the assessed text self-contained, information?
Assessed Answer: No





'\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: ${answer}\n\n---\n\nQuestion: At My Window was released by which American singer-songwriter?\nAnswer:\x1b Bob Dylan\x1b\n\n\n\n\n\nAssess the quality of a tweet along the specified dimension.\n\n---\n\nFollow the following format.\n\nAssessed Text: ${assessed_text}\nAssesed Question: ${assesed_question}\nAssessed Answer: Yes or Not\n\n---\n\nAssessed Text: Bob Dylan\nAssesed Question: The text should answer `At My Window was released by which American singer-songwriter?` with `John Townes Van Zandt`. Does the assessed text contain this answer?\nAssessed Answer:\x1b No\x1b\n\n\n\n\n\nAssess the quality of a tweet along the specified dimension.\n\n---\n\nFollow the following format.\n\nAssessed Text: ${assessed_text}\nAssesed Question: ${assesed_question}\nAssessed Answer: Yes or Not\n\n---\n\nAssessed Text: Bob Dylan\nAssesed Question: Is the assessed

### Optimizers

In [77]:
from dspy import context, Module, settings, ChainOfThought, OpenAI
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate import Evaluate
from dspy.datasets import HotPotQA
from dspy.datasets.gsm8k import GSM8K
from dspy.datasets.gsm8k import parse_integer_answer, gsm8k_metric
from rich import print
import inspect


from rich import print
from dotenv import dotenv_values

secret = dotenv_values('../../.secret')
gpt_turbo = OpenAI('gpt-3.5-turbo', api_key=secret['OPEN_AI_API_KEY'])
settings.configure(lm=gpt_turbo)


In [35]:
gsm8k = GSM8K()
train, dev = gsm8k.train, gsm8k.dev

100%|██████████| 1319/1319 [00:00<00:00, 72539.36it/s]


In [37]:
hpqa = HotPotQA(train_seed=1, train_size=10, eval_seed=42, dev_size=10)


/Users/beltre.wilton/miniforge3/envs/tars_env/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [39]:
class CoT(Module):
    def __init__(self):
        super().__init__()
        self.prog = ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)


cot = CoT()


In [45]:
print(inspect.getsource(gsm8k_metric))

def gsm8k_metric(gold, pred, trace=None):
    return int(parse_integer_answer(str(gold.answer))) == int(parse_integer_answer(str(pred.answer)))

In [46]:
print(inspect.getsource(parse_integer_answer))

def parse_integer_answer(answer, only_first_line=True):
    try:
        if only_first_line:
            answer = answer.strip().split('\n')[0]

        # find the last token that has a number in it
        answer = [-1]
        answer = answer.split('.')[0]
        answer = ''.join()
        answer = int(answer)

    except (ValueError, IndexError):
        # print(answer)
        answer = 0
    
    return answer

In [70]:
dev_fisrt_3 = [x.with_inputs('question') for x in dev[:3]]
dev_evaluator = Evaluate(devset=dev_fisrt_3, num_threads=4, metric=gsm8k_metric, display=True, display_progress=True)

In [71]:
dev_fisrt_3

[Example({'question': '20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in one season, which is 50 miles apart, then the next season they fly from lake Disney to lake London, 60 miles apart, calculate the combined distance all of the birds have traveled in the two seasons.', 'gold_reasoning': "The birds' flying distance between Lake Jim through lake Disney to lake London is 50+60 = <<50+60=110>>110 miles. Since each bird flies the 110 miles distance while migrating, the combined distance they fly together is 110*20 = <<110*20=2200>>2200 miles.", 'answer': '2200'}) (input_keys={'question'}),
 Example({'question': 'Wendy went to the dentist for a cleaning, two fillings, and a tooth extraction. The dentist charges $70 for a cleaning and $120 for a filling. Wendy’s dentist bill was five times the cost of a filling. What did Wendy pay for the tooth extraction?', 'gold_reasoning': 'Wendy’s dentist bill was 5 * $120 = 

In [72]:
run_dev_val = dev_evaluator(cot)

Average Metric: 0 / 3  (0.0): 100%|██████████| 3/3 [00:00<00:00, 2631.31it/s]


In [78]:
gpt_turbo.inspect_history(n=12)

''

In [73]:
run_dev_val

0.0

In [74]:
cot.save('cot_baseline.json')


In [75]:
optim = BootstrapFewShot(
    metric=gsm8k_metric,
    max_bootstrapped_demos=5,
    max_labeled_demos=3
)

In [79]:
compile_cot = optim.compile(cot,
                            trainset=train[:4],
                            valset=dev_fisrt_3)

100%|██████████| 4/4 [00:07<00:00,  1.99s/it]


In [81]:
len(gpt_turbo.history)

5

In [83]:
gpt_turbo.inspect_history(n=5)




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Sarah buys 20 pencils on Monday. Then she buys 18 more pencils on Tuesday. On Wednesday she buys triple the number of pencils she did on Tuesday. How many pencils does she have?
Answer: 92

---

Question: Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.
Answer: 600000

---

Question: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?
Reasoning: Let's think step by step in order to Answer: 28





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: $

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Sarah buys 20 pencils on Monday. Then she buys 18 more pencils on Tuesday. On Wednesday she buys triple the number of pencils she did on Tuesday. How many pencils does she have?\nAnswer: 92\n\n---\n\nQuestion: Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.\nAnswer: 600000\n\n---\n\nQuestion: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?\nReasoning: Let's think step by step in order to\x1b Answer: 28\x1b\n\n\n\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---

In [ ]:
1
4
5


In [84]:
compile_cot.save('cot_compiled.json')

In [85]:

inventory = []
for i in range(3, 1000):
    inventory.append({'product': chr(ord('A') + i), 'quantity': i})

In [90]:
chr(ord('A') + 1)

'B'

In [92]:
gpt_turbo.inspect_history()




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.
Answer: 600000

---

Question: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?
Answer: 24

---

Question: Sarah buys 20 pencils on Monday. Then she buys 18 more pencils on Tuesday. On Wednesday she buys triple the number of pencils she did on Tuesday. How many pencils does she have?
Reasoning: Let's think step by step in order to produce the answer. We first add the number of pencils she bought on Monday and Tuesday, which is 20 + 18 = 38 pencils. T

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.\nAnswer: 600000\n\n---\n\nQuestion: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?\nAnswer: 24\n\n---\n\nQuestion: Sarah buys 20 pencils on Monday. Then she buys 18 more pencils on Tuesday. On Wednesday she buys triple the number of pencils she did on Tuesday. How many pencils does she have?\nReasoning: Let's think step by step in order to\x1b produce the answer. We first add the number of pencils she bought on Monday and Tuesday, w